In [239]:
import utility
import pandas as pd
import urllib.request
import json 
from bs4 import BeautifulSoup
import requests
import json
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import datetime, timedelta, date
import pytz
import dateutil.parser
from collections import defaultdict
import random
import json
import csv
from dateutil.parser import parse
import os
import datetime

In [240]:
project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/"
file_path = "project/data_extraction/data/unannotated_data/cbc/interestrates_CBC_article.json"
converted_df = utilities.convert_json_to_df(project_path, file_path)

In [241]:
converted_df.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,Pete Evans,The head of the Federal Reserve said Wednesday...,U.S. president goes negative on Jerome Powell ...,"[May 13, 2020 10:48 AM ET, 2020-05-13T15:17:41...",CBC News,U.S. Fed chair rules out negative interest rat...,http://www.cbc.ca/news/business/powel-trump-ne...,https://i.cbc.ca/1.5258204.1566589338!/fileIma...
1,None,None,None,None,None,None,http://www.cbc.ca/player/play/1707273795558,None
2,None,None,None,None,None,None,http://www.cbc.ca/player/play/1706560067565,None
3,Pete Evans,Scotiabank posted a profit Tuesday morning of ...,1st of big Canadian banks to report in pandemi...,"[May 26, 2020 8:25 AM ET, 2020-05-26T14:31:07....",CBC News,Scotiabank profit falls 40% as bad loans more ...,http://www.cbc.ca/news/business/scotiabank-ear...,https://i.cbc.ca/1.5291880.1569008578!/cpImage...
4,None,None,None,None,None,None,http://www.cbc.ca/news/canada/british-columbia...,None


In [253]:
def get_unannotated_data(combined_df, annotated_df, indicator, source, prediction_file_path = "project/sentiment_analyzer/data/predictions_data/"):
    '''get unannotated data (remove annotated data from all the articles that are collected)'''
    # Running this line will erase current data
    #prediction_file_path = "../../sentiment_analyzer/data/predictions_data/bloomberg/" 
    # try this line for testing
    total = combined_df
    drop_list = annotated_df.index.values.tolist()
    total = total.drop(drop_list)
    total['title_desc'] = total['title'] + '. ' + total['description']
    total = total[['source', 'title_desc', 'publishedAt']]
    total = total.drop_duplicates(subset='title_desc', keep='first' )
    if utility.check_dir_exists(prediction_file_path + source):
        try:
            total.to_csv(prediction_file_path + source + '/predictions_dataset_' + indicator + "_" + source + '.csv')
        except:
            print("unable to write to file")
            return False
    if utility.file_exists(prediction_file_path +  source + '/predictions_dataset_' + indicator + "_" + source + '.csv'):
        return True
    else:
        print("Final directory is not created")
        return False


In [247]:

def write_df_to_csv(df,project_path,file_path,file_name):
    """Takes a dataframe and writes to a file"""
    if os.path.isdir(project_path+file_path):
        df.to_csv(project_path+file_path+file_name, encoding='utf-8', index=False)
        print(project_path+file_path+file_name, "successfully written")
    else:
        print("Path does not exist:", project_path+file_path)
        return None

In [256]:
def cbc_sampling_wrapper(economic_indicator,project_path,json_file_path, column_names_list, sample_size,source_string, remove_Nans = True,output_file_path="project/data_extraction/data/annotated_data/"):
    """This function creates a sample from a given file  of json 
    Input:
    ------
    economic_indicator: The economic indicator
    project_path: The path of the project
    json_file_path: The path tto the file which has the article jsons
    column_names_list: The list of column names to retain in the final sampled file
    sample_size: The number of articles to sample per month in any year
    source_string: The source (CBC/Bloomberg)"""
    try:
        df = utility.convert_json_to_df(project_path, json_file_path)
        preprocessed_df = utility.preprocess_df(df,column_names_list)
        preprocessed_df["source"] = source_string
        preprocessed_df = utility.apply_lambda(preprocessed_df,'publishedAt' ,"lambda x: x[1]")
        sampled_df = utility.sample_dataframe_by_month(preprocessed_df, sample_size)
        sampled_df = utility.apply_lambda(sampled_df,'publishedAt',"lambda x: dateutil.parser.isoparse(x).date()")
        # Write sampled dataframe to a different file in a different path
        write_df_to_csv(sampled_df,project_path,output_file_path+source_string,economic_indicator+"_sample.csv")
        if os.path.exists(project_path + output_file_path +source_string+"/"+economic_indicator+"_sample.csv"):
            get_unannotated_data(preprocessed_df, sampled_df, economic_indicator, source_string, project_path+output_file_path)
            print("Sample and predictions ready")
            return True
            
        
        else:
            print("SOmething went wrong")
            
            
    except:
        print("Could not successfully sample the data")
        return False
    
project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/"
json_file_path = "project/data_extraction/data/unannotated_data/cbc/mortgagerates_CBC_article.json"
c = cbc_sampling_wrapper("mortgagerates",project_path,json_file_path,['title', 'description','publishedAt'], 2,"cbc")

/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/data_extraction/data/annotated_data/cbcmortgagerates_sample.csv successfully written
Sample and predictions ready
